In [1]:
import pandas as pd
import glob
import numpy as np

In [2]:
def type_test(df, col):
    try:
        menor_tipo = df[col].dtype
    except KeyError:
        raise ValueError
    menor_memo = df[col].memory_usage()

    # int 
    if df[col].isna().sum() == 0:
        try:
            aux = pd.to_numeric(df[col], downcast='integer')

            a = aux.memory_usage()

            if menor_memo > a:
                menor_tipo = aux.dtype
                menor_memo = a
        except ValueError:
            # print(f'Coluna {col} - int value error')
            pass

    # category
    try: 
        aux = df[col].astype("category")

        a = aux.memory_usage()

        if menor_memo > a:
            menor_tipo = aux.dtype
            menor_memo = a
    except ValueError:
        # print(f'Coluna {col} - cal value error')
        pass

    # float
    try:
        aux = pd.to_numeric(df[col], downcast='float')

        a = aux.memory_usage()

        if menor_memo > a:
            menor_tipo = aux.dtype
            menor_memo = a
    except ValueError:
        # print(f'Coluna {col} - float value error')
        pass

    # return menor_tipo
    if menor_tipo == 'category':
        df[col] = df[col].astype(str).astype('category')
    else:
        df[col] = df[col].astype(menor_tipo)

In [26]:
files = glob.glob('SINASC/*parquet')
files

['SINASC\\DN2012.parquet',
 'SINASC\\DN2013.parquet',
 'SINASC\\DN2014.parquet',
 'SINASC\\DN2015.parquet',
 'SINASC\\DN2016.parquet',
 'SINASC\\DN2017.parquet',
 'SINASC\\DN2018.parquet',
 'SINASC\\DN2019.parquet',
 'SINASC\\DN2020.parquet',
 'SINASC\\DN2021.parquet',
 'SINASC\\DN2022.parquet']

---

## Tipos

In [40]:
ano = 13
for item in files[1:]:
    df = pd.read_parquet(item)

    for i in df.columns:
        type_test(df, i)

    df.to_parquet(f'test/DN20{ano}.parquet')
    ano+=1

## Concatenação

In [47]:
files = glob.glob('test/*parquet')
files

['test\\DN2012.parquet',
 'test\\DN2013.parquet',
 'test\\DN2014.parquet',
 'test\\DN2015.parquet',
 'test\\DN2016.parquet',
 'test\\DN2017.parquet',
 'test\\DN2018.parquet',
 'test\\DN2019.parquet',
 'test\\DN2020.parquet',
 'test\\DN2021.parquet',
 'test\\DN2022.parquet']

#### 2012 - 2017

In [51]:
df = pd.DataFrame()

for item in files[:6]:
    aux = pd.read_parquet(item)

    df = pd.concat([aux, df], axis=0)

In [55]:
df.memory_usage().sum()

10239002904

In [57]:
for i in df.columns:
    type_test(df, i)

In [58]:
df.memory_usage().sum()

2064750408

In [59]:
df.to_parquet('DNParte1.parquet')

#### 2018 - 2022

In [67]:
df = pd.DataFrame()

for item in files[6:]:
    aux = pd.read_parquet(item)

    df = pd.concat([aux, df], axis=0)

In [68]:
df.memory_usage().sum()

4299572311

In [69]:
for i in df.columns:
    type_test(df, i)

In [70]:
df.memory_usage().sum()

1355103377

In [71]:
df.to_parquet('DNParte2.parquet')

## Concatenação Final

In [82]:
df = pd.read_parquet('DNParte1.parquet')

In [84]:
df.memory_usage().sum()

2064750408

In [88]:
aux = pd.read_parquet('DNParte2.parquet')

In [89]:
aux.memory_usage().sum()

1355103377

In [92]:
df = pd.concat([aux, df], axis=0)

In [93]:
df.memory_usage().sum()

17163594340

In [96]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34547849 entries, 2220000001 to 2123197707
Data columns (total 68 columns):
 #   Column      Dtype   
---  ------      -----   
 0   ORIGEM      int8    
 1   CODESTAB    object  
 2   CODMUNNASC  object  
 3   LOCNASC     int8    
 4   IDADEMAE    object  
 5   ESTCIVMAE   object  
 6   ESCMAE      object  
 7   CODOCUPMAE  object  
 8   QTDFILVIVO  object  
 9   QTDFILMORT  object  
 10  CODMUNRES   object  
 11  GESTACAO    object  
 12  GRAVIDEZ    object  
 13  PARTO       object  
 14  CONSULTAS   object  
 15  DTNASC      object  
 16  HORANASC    object  
 17  SEXO        object  
 18  APGAR1      object  
 19  APGAR5      object  
 20  RACACOR     object  
 21  PESO        object  
 22  IDANOMAL    object  
 23  DTCADASTRO  object  
 24  CODANOMAL   object  
 25  NUMEROLOTE  object  
 26  VERSAOSIST  object  
 27  DTRECEBIM   object  
 28  DIFDATA     object  
 29  DTRECORIGA  object  
 30  NATURALMAE  object  
 31  CODMUNNATU  obje

In [98]:
for i in df.columns:
    type_test(df, i)

In [99]:
df.to_parquet('DN.parquet')

In [102]:
df.memory_usage().sum()

3915490553

In [104]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34547849 entries, 2220000001 to 2123197707
Data columns (total 68 columns):
 #   Column      Dtype   
---  ------      -----   
 0   ORIGEM      int8    
 1   CODESTAB    category
 2   CODMUNNASC  category
 3   LOCNASC     int8    
 4   IDADEMAE    category
 5   ESTCIVMAE   category
 6   ESCMAE      category
 7   CODOCUPMAE  category
 8   QTDFILVIVO  category
 9   QTDFILMORT  category
 10  CODMUNRES   category
 11  GESTACAO    category
 12  GRAVIDEZ    category
 13  PARTO       category
 14  CONSULTAS   category
 15  DTNASC      category
 16  HORANASC    category
 17  SEXO        category
 18  APGAR1      category
 19  APGAR5      category
 20  RACACOR     category
 21  PESO        category
 22  IDANOMAL    category
 23  DTCADASTRO  category
 24  CODANOMAL   category
 25  NUMEROLOTE  category
 26  VERSAOSIST  category
 27  DTRECEBIM   category
 28  DIFDATA     category
 29  DTRECORIGA  category
 30  NATURALMAE  category
 31  CODMUNNATU  cate

---

## Outros

In [11]:
files[:6]

['SINASC\\DN2012.parquet',
 'SINASC\\DN2013.parquet',
 'SINASC\\DN2014.parquet',
 'SINASC\\DN2015.parquet',
 'SINASC\\DN2016.parquet',
 'SINASC\\DN2017.parquet']

In [9]:
files[6:]

['SINASC\\DN2018.parquet',
 'SINASC\\DN2019.parquet',
 'SINASC\\DN2020.parquet',
 'SINASC\\DN2021.parquet',
 'SINASC\\DN2022.parquet']

In [27]:
df = pd.DataFrame()

for item in files[:6]:
    aux = pd.read_parquet(item)
    antes = aux.memory_usage().sum()

    for column in aux.columns:
        type_test(aux, column)
    
    print(f'{item} - {len(aux)} | Uso da memória: Antes {antes} - Depois {aux.memory_usage().sum()}')
    df = pd.concat([aux, df], axis=0)

SINASC\DN2012.parquet - 3197707 | Uso da memória: Antes 1458154392 - Depois 282886693
SINASC\DN2013.parquet - 3197575 | Uso da memória: Antes 1717097775 - Depois 302435251
SINASC\DN2014.parquet - 3282309 | Uso da memória: Antes 1818399186 - Depois 294146378
SINASC\DN2015.parquet - 3311296 | Uso da memória: Antes 1642402816 - Depois 293330200
SINASC\DN2016.parquet - 3149750 | Uso da memória: Antes 1562276000 - Depois 279052292


C:\Users\ricmo\AppData\Local\Temp\ipykernel_3852\1703892524.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([aux, df], axis=0)


SINASC\DN2017.parquet - 3216411 | Uso da memória: Antes 1595339856 - Depois 284980940


C:\Users\ricmo\AppData\Local\Temp\ipykernel_3852\1703892524.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([aux, df], axis=0)


In [53]:
df.loc[df['SEXO'] == 'F', 'SEXO'] = 2
df.loc[df['SEXO'] == 'M', 'SEXO'] = 1
df.loc[df['SEXO'] == 'I', 'SEXO'] = 0

df['SEXO'].info()

<class 'pandas.core.series.Series'>
Index: 19355048 entries, 2170000001 to 2123197707
Series name: SEXO
Non-Null Count     Dtype   
--------------     -----   
19355048 non-null  category
dtypes: category(1)
memory usage: 166.1 MB


In [61]:
df['SEXO'] = df['SEXO'].astype(np.int8)

In [29]:
for column in df.columns:
    type_test(df, column)

ArrowInvalid: ("Could not convert 'F' with type str: tried to convert to int64", 'Conversion failed for column SEXO with type category')

In [65]:
df.to_parquet('DN1.parquet')

---

In [7]:
df = pd.DataFrame()

for item in files[6:]:
    aux = pd.read_parquet(item)
    antes = aux.memory_usage().sum()

    for column in aux.columns:
        type_test(aux, column)
    
    print(f'{item} - {len(aux)} | Uso da memória: Antes {antes} - Depois {aux.memory_usage().sum()}')
    df = pd.concat([aux, df], axis=0)

SINASC\DN2018.parquet - 3232183 | Uso da memória: Antes 1793861565 - Depois 289673928
SINASC\DN2019.parquet - 3139558 | Uso da memória: Antes 1742454690 - Depois 281426984
SINASC\DN2020.parquet - 3013130 | Uso da memória: Antes 1672287150 - Depois 270193584
SINASC\DN2021.parquet - 2951464 | Uso da memória: Antes 1638062520 - Depois 264721866
SINASC\DN2022.parquet - 2856466 | Uso da memória: Antes 1585338630 - Depois 256229037


In [8]:
for column in df.columns:
    type_test(df, column)

In [9]:
df.to_parquet('DN2.parquet')

---

In [16]:
df = pd.read_parquet('DN1.parquet')

ArrowMemoryError: malloc of size 774201920 failed

In [ ]:
aux = pd.read_parquet('DN2.parquet')

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19355048 entries, 2170000001 to 2123197707
Data columns (total 67 columns):
 #   Column      Dtype   
---  ------      -----   
 0   contador    int32   
 1   ORIGEM      int64   
 2   CODESTAB    float64 
 3   CODMUNNASC  int64   
 4   LOCNASC     int64   
 5   IDADEMAE    float64 
 6   ESTCIVMAE   float64 
 7   ESCMAE      float64 
 8   CODOCUPMAE  float64 
 9   QTDFILVIVO  float64 
 10  QTDFILMORT  float64 
 11  CODMUNRES   int64   
 12  GESTACAO    float64 
 13  GRAVIDEZ    float64 
 14  PARTO       float64 
 15  CONSULTAS   float64 
 16  DTNASC      int64   
 17  HORANASC    float64 
 18  SEXO        int8    
 19  APGAR1      float64 
 20  APGAR5      float64 
 21  RACACOR     float64 
 22  PESO        float64 
 23  IDANOMAL    float64 
 24  DTCADASTRO  float64 
 25  CODANOMAL   category
 26  NUMEROLOTE  float64 
 27  VERSAOSIST  category
 28  DTRECEBIM   float64 
 29  DIFDATA     float64 
 30  DTRECORIGA  float64 
 31  NATURALMAE  floa

In [20]:
aux.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15192801 entries, 2220000001 to 2183232183
Data columns (total 61 columns):
 #   Column      Dtype   
---  ------      -----   
 0   ORIGEM      int64   
 1   CODESTAB    float64 
 2   CODMUNNASC  int64   
 3   LOCNASC     int8    
 4   IDADEMAE    float64 
 5   ESTCIVMAE   float64 
 6   ESCMAE      float64 
 7   CODOCUPMAE  float64 
 8   QTDFILVIVO  float64 
 9   QTDFILMORT  float64 
 10  CODMUNRES   int64   
 11  GESTACAO    float64 
 12  GRAVIDEZ    float64 
 13  PARTO       float64 
 14  CONSULTAS   float64 
 15  DTNASC      int64   
 16  HORANASC    float64 
 17  SEXO        int8    
 18  APGAR1      float64 
 19  APGAR5      float64 
 20  RACACOR     float64 
 21  PESO        float64 
 22  IDANOMAL    float64 
 23  DTCADASTRO  int64   
 24  CODANOMAL   category
 25  NUMEROLOTE  float64 
 26  VERSAOSIST  category
 27  DTRECEBIM   float64 
 28  DIFDATA     int64   
 29  DTRECORIGA  float64 
 30  NATURALMAE  float64 
 31  CODMUNNATU  floa

In [22]:
for column in df.columns:
    type_test(df, column)

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19355048 entries, 2170000001 to 2123197707
Data columns (total 67 columns):
 #   Column      Dtype   
---  ------      -----   
 0   contador    int32   
 1   ORIGEM      int8    
 2   CODESTAB    category
 3   CODMUNNASC  category
 4   LOCNASC     int8    
 5   IDADEMAE    category
 6   ESTCIVMAE   category
 7   ESCMAE      category
 8   CODOCUPMAE  category
 9   QTDFILVIVO  category
 10  QTDFILMORT  category
 11  CODMUNRES   category
 12  GESTACAO    category
 13  GRAVIDEZ    category
 14  PARTO       category
 15  CONSULTAS   category
 16  DTNASC      category
 17  HORANASC    category
 18  SEXO        int8    
 19  APGAR1      category
 20  APGAR5      category
 21  RACACOR     category
 22  PESO        category
 23  IDANOMAL    category
 24  DTCADASTRO  category
 25  CODANOMAL   category
 26  NUMEROLOTE  category
 27  VERSAOSIST  category
 28  DTRECEBIM   category
 29  DIFDATA     category
 30  DTRECORIGA  category
 31  NATURALMAE  cate

In [26]:
for column in aux.columns:
    type_test(aux, column)

In [27]:
aux.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15192801 entries, 2220000001 to 2183232183
Data columns (total 61 columns):
 #   Column      Dtype   
---  ------      -----   
 0   ORIGEM      int8    
 1   CODESTAB    category
 2   CODMUNNASC  category
 3   LOCNASC     int8    
 4   IDADEMAE    category
 5   ESTCIVMAE   category
 6   ESCMAE      category
 7   CODOCUPMAE  category
 8   QTDFILVIVO  category
 9   QTDFILMORT  category
 10  CODMUNRES   category
 11  GESTACAO    category
 12  GRAVIDEZ    category
 13  PARTO       category
 14  CONSULTAS   category
 15  DTNASC      category
 16  HORANASC    category
 17  SEXO        int8    
 18  APGAR1      category
 19  APGAR5      category
 20  RACACOR     category
 21  PESO        category
 22  IDANOMAL    category
 23  DTCADASTRO  category
 24  CODANOMAL   category
 25  NUMEROLOTE  category
 26  VERSAOSIST  category
 27  DTRECEBIM   category
 28  DIFDATA     int16   
 29  DTRECORIGA  category
 30  NATURALMAE  category
 31  CODMUNNATU  cate

In [30]:
df = pd.concat([aux, df], axis=0)

C:\Users\ricmo\AppData\Local\Temp\ipykernel_17004\2452581115.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([aux, df], axis=0)


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34547849 entries, 2220000001 to 2123197707
Data columns (total 68 columns):
 #   Column      Dtype   
---  ------      -----   
 0   ORIGEM      int8    
 1   CODESTAB    float64 
 2   CODMUNNASC  int64   
 3   LOCNASC     int8    
 4   IDADEMAE    category
 5   ESTCIVMAE   category
 6   ESCMAE      float64 
 7   CODOCUPMAE  float64 
 8   QTDFILVIVO  float64 
 9   QTDFILMORT  category
 10  CODMUNRES   int64   
 11  GESTACAO    category
 12  GRAVIDEZ    category
 13  PARTO       category
 14  CONSULTAS   category
 15  DTNASC      int64   
 16  HORANASC    float64 
 17  SEXO        int8    
 18  APGAR1      float64 
 19  APGAR5      category
 20  RACACOR     float64 
 21  PESO        float64 
 22  IDANOMAL    category
 23  DTCADASTRO  float64 
 24  CODANOMAL   object  
 25  NUMEROLOTE  float64 
 26  VERSAOSIST  object  
 27  DTRECEBIM   float64 
 28  DIFDATA     float64 
 29  DTRECORIGA  category
 30  NATURALMAE  float64 
 31  CODMUNNATU  floa

In [34]:
for column in df.columns:
    type_test(df, column)

In [35]:
df.to_parquet('DN.parquet', engine='fastparquet')

---

In [3]:
df = pd.read_parquet('DN.parquet')

ArrowMemoryError: malloc of size 16306584768 failed

In [ ]:
df.info()